In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import sys

ROOT_DIR = '../'
sys.path.insert(1, '../production_code/')
from constants import *

In [2]:
# importing data
accidents = pd.read_csv(ROOT_DIR + ACCIDENT_DATA_GENERAL_DIR)
node = pd.read_csv(ROOT_DIR + ACCIDENT_DATA_NODE_DIR)
person = pd.read_csv(ROOT_DIR + ACCIDENT_DATA_PERSON_DIR)
atmospheric = pd.read_csv(ROOT_DIR + ACCIDENT_DATA_ATMOSPHERIC_DIR)
road_cond = pd.read_csv(ROOT_DIR + ACCIDENT_DATA_ROAD_COND_DIR)

C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (6,9,10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# data model

features

| group name | column name | data type | description | original dataset |
| ---- | ----- | ---- | ---- | ---- |
| time | date | pd.datetime |  | accident
|  | day of week | pd.datetime |  | accident
|  | time of day (hour) | pd.datetime |  | accident
| | light level | int | dark (any) = 0, dawn/dusk = 1, day = 2 | LIGHT_COND, accident
| | | | | 
| location | node_id | int | | node |
| | lga | string | local area | node |
| | region | string | | node |
| | long | float | longitude | node |
| | lat | float | latitude | node |
| |  | | |
| atmospheric | 1: clear         | bool |            | atmostpheric
|             | 2: raining       | bool |            | atmostpheric
|             | 3: snowing       | bool |            | atmostpheric
|             | 4: fog           | bool |            | atmostpheric
|             | 5: smoke         | bool |            | atmostpheric
|             | 6: dust          | bool |            | atmostpheric
|             | 7: winds         | bool |            | atmostpheric
|             | 9: unknown       | bool | remove unknown | atmostpheric
| | | | |
| road_cond   | 1: dry           | bool |           | road condition
|             | 2: wet           | bool |           | road condition
|             | 3: muddy         | bool |           | road condition
|             | 4: snowy         | bool |           | road condition
|             | 5: icy           | bool |           | road condition
|             | 9: unknown       | bool | remove unknown  | road condition
| | | | |

labels

| column name | data type | description | original dataset |
| ----- | ---- | ---- | ---- |
| police_needed | int | number of police for colision bin | POLICE_ATTENDED, accident |
| ambulance_needed | int | number of ambulance needed for that colision bin | to investigate, mix of TAKEN_HOSPITAL in PERSON and if injuries are serious in ACCIDENT | 


 


 ### Abulance needed

assuming an ambulace is called if

there are more than 
- 5 people involved in a crash ? (didnt include)
- any person in the crash has an inj_level > 0
- any person was taken to the hospital

only calls one ambulance no matter number of people, to keep inline with number of police



In [3]:
accidents.groupby('ACCIDENT_NO')['ACCIDENTDATE'].count().sort_values()   # no duplicates for accidnets no, theyre unique


node.groupby('ACCIDENT_NO')['NODE_ID'].count().sort_values()
node.query('ACCIDENT_NO == "T20170021373"')    # dueplicated nodes per accident are due to issues with postcode double ups, not in multi location



person.groupby('ACCIDENT_NO')['PERSON_ID'].count().sort_values()     # duplicate ACCIDENT_NO for multiple perople in single colision
person.query('ACCIDENT_NO == "T20130018492"')   

atmospheric.groupby('ACCIDENT_NO')['ATMOSPH_COND'].count().sort_values()     
atmospheric.query('ACCIDENT_NO == "T20190001830"')           # several conditions like winds and rain, need to factor in, probably pivot as seperate columns



road_cond.groupby('ACCIDENT_NO')['SURFACE_COND'].count().sort_values()
road_cond.query('ACCIDENT_NO == "T20070019368"')          # same as atmospheric, several conditions can be met

,ACCIDENT_NO,SURFACE_COND,Surface Cond Desc,SURFACE_COND_SEQ
19319,T20070019368,2,Wet,3
19320,T20070019368,4,Snowy,1
19321,T20070019368,5,Icy,2


# filtering and transforming 

### accidents

In [4]:
# changing type
accidents['ACCIDENT_NO'] = accidents['ACCIDENT_NO'].astype(str)

In [5]:
# date

# removing na dates and times
accidents = accidents.dropna(subset=['ACCIDENTDATE'])
accidents = accidents.dropna(subset=['ACCIDENTTIME'])

# only keeping more recent data from 2016 prior, new data will be more accurate
accidents = accidents[pd.to_datetime(accidents['ACCIDENTDATE']) > pd.to_datetime(EARLIEST_DATE)].reset_index(drop = True)
accidents.head(3)

# combine date strings
accidents.loc[:,'date'] = pd.to_datetime(accidents[['ACCIDENTDATE','ACCIDENTTIME']].apply(lambda x: x[0] + " " + x[1] , axis = 1))


In [6]:
# adding more date breakdowns for testing
accidents.loc[:,'day'] = accidents.loc[:,'date'].dt.dayofweek 
accidents.loc[:,'hour'] = accidents.loc[:,'date'].dt.hour 
accidents.loc[:,'hour_bin'] = pd.cut(accidents.loc[:,'hour'], [0, 6, 12, 18, 24] , labels=[0,1,2,3], right=False).cat.codes
accidents.loc[:,'season'] = accidents.loc[:,'date'].dt.month % 12 // 3 + 1
accidents.loc[:,'month'] = accidents.loc[:,'date'].dt.month
accidents.loc[:,'year'] = accidents.loc[:,'date'].dt.year
accidents.loc[:,'date_stamp'] = accidents.loc[:,'date'].dt.date


In [7]:
# light conditions

# dont remove unknown lighting conditions, didnt end up using it
# accidents = accidents.query("LIGHT_CONDITION != 9")

# converts conditions to 0-2 scale of daylight
accidents.loc[:,'day_light'] = accidents['LIGHT_CONDITION'].apply(lambda light_level: 3 - min(light_level, 3))    

# printing
accidents.groupby('LIGHT_CONDITION')['day_light'].unique()

LIGHT_CONDITION
1    [2]
2    [1]
3    [0]
4    [0]
5    [0]
6    [0]
9    [0]
Name: day_light, dtype: object

In [8]:
# removing when unsure if police attened or not, didnt remove many rows
accidents = accidents.query("POLICE_ATTEND != 9")   

# shifts police attened to boolean yes or no
accidents.loc[:,'police_needed'] = accidents['POLICE_ATTEND'].apply(lambda x: 2 - x)

# printing
accidents.groupby('POLICE_ATTEND')['police_needed'].unique()

POLICE_ATTEND
1    [1]
2    [0]
Name: police_needed, dtype: object

In [9]:
# columns of interest
accidents_of_interest = ['date','date_stamp', 'day', 'hour', 'hour_bin', 'season', 'month', 'day_light']
accidents_label_columns = ['police_needed']
id_columns = ['ACCIDENT_NO']

# printing data
accidents[id_columns + accidents_of_interest + accidents_label_columns].head(3)

,ACCIDENT_NO,date,date_stamp,day,hour,hour_bin,season,month,day_light,police_needed
0,T20150000056,2015-02-01 03:00:00,2015-02-01,6,3,0,1,2,0,1
1,T20150000060,2015-02-01 07:30:00,2015-02-01,6,7,1,1,2,2,1
2,T20150000063,2015-02-01 08:30:00,2015-02-01,6,8,1,1,2,2,1


### person

In [10]:
# only keeping instances with ids that exist
# person = person[person['ACCIDENT_NO'].isin(accidents['ACCIDENT_NO'])]

# converting injury level to number
person.loc[:,'injury_level'] = person['INJ_LEVEL'].apply(pd.to_numeric, errors= 'coerce')
person = person.dropna(subset = ['injury_level'])

# calculating if individual needed an ambulance
person.loc[:,'ambulance_needed'] = person[['injury_level','TAKEN_HOSPITAL']].apply(lambda x: (x['injury_level'] < 4) or (x['TAKEN_HOSPITAL'] == 'Y'), axis = 1)    

# person.groupby(['INJ_LEVEL'])['ambulance_needed'].unique()   # data looks good

# accidents where ambulance was needed
person_grouped = person.groupby('ACCIDENT_NO')['ambulance_needed'].any()

# pivots to summarize ambulance data per crash
person_pivotted = person\
    .pivot_table(index = "ACCIDENT_NO", values = 'ambulance_needed', aggfunc = "max")\
    .fillna(0)\
    .applymap(lambda x: min(x, 1))\
    .reset_index()

# of interest columns
person_of_interest = []
person_label_columns = ['ambulance_needed']

# prints data
person_pivotted[id_columns + person_of_interest + person_label_columns].head(3)

,ACCIDENT_NO,ambulance_needed
0,T20060000010,True
1,T20060000018,True
2,T20060000022,True


### node

In [11]:
# only keeping instances with ids that exist
print("initial shape: " + str(node.shape))
# node = node[node['ACCIDENT_NO'].isin(accidents['ACCIDENT_NO'])]

# reanaming columns
node = node.rename(columns = {
    'REGION_NAME': 'region', 
    'LGA_NAME':'lga', 
    'NODE_ID':'node_id',
    'Lat':'lat',
    'Long':'long'
    })

# stringifying
node['region'] = node['region'].astype(str)
node['lga'] = node['lga'].astype(str)

# to numeric
node.loc[:,'lat'] = node.loc[:,'lat'].apply(pd.to_numeric)
node.loc[:,'long'] = node.loc[:,'long'].apply(pd.to_numeric)

# removing blank regions
node = node.query('region != " "')

# printing shape to check for issues
print("final shape: " + str(node.shape))

# setting of interst and printing
node_of_interest = ['node_id','lga','region','lat','long']
node[id_columns + node_of_interest].drop_duplicates().head(3)

initial shape: (221797, 12)
final shape: (221749, 12)


,ACCIDENT_NO,node_id,lga,region,lat,long
0,T20060002689,22,MELBOURNE,METROPOLITAN NORTH WEST REGION,-37.796596,144.951197
1,T20060010827,22,MELBOURNE,METROPOLITAN NORTH WEST REGION,-37.796596,144.951197
2,T20060017279,22,MELBOURNE,METROPOLITAN NORTH WEST REGION,-37.796596,144.951197


### road_cond

after inital testing, simplified data as road condition tended not to be used that much and was mostly inaccuate

In [12]:
# road condition dictionary
road_cond_id_to_desc = {
    1: 'dry',
    2: 'wet',
    3: 'muddy',
    4: 'snowy',
    5: 'icy',
    9: 'not known',
}

# simplify road conditions to dry or not
def simpilfy_road_cond(id):
    if id  == 1:   # dry
        return 1   # road not dry
    else:
        return 0   # road not dry

In [13]:
print("initial shape: " + str(road_cond.shape))

# only keeping instances with ids that exist, didnt need
# road_cond = road_cond[road_cond['ACCIDENT_NO'].isin(accidents['ACCIDENT_NO'])]


# removes unknowns,
# road_cond = road_cond.query("SURFACE_COND != 9")   # didnt need in the end
road_cond = road_cond.dropna(subset = ['SURFACE_COND'])

# simplifying conditions to either dry or not
road_cond.loc[:,'SURFACE_COND'] = road_cond.loc[:,'SURFACE_COND'].apply(lambda x: x == 1)

# OLD CODE: instead of simplifying conditions to dry or not, was breaking down into several options
# simplifying descriptions
# road_cond.loc[:,'Atmosph Cond Desc'] = road_cond.loc[:,'ATMOSPH_COND'].apply(lambda id: road_cond_id_to_desc[id])
#
# making descrions lowercase
# road_cond.loc[:,'Surface Cond Desc'] = road_cond.loc[:,'Surface Cond Desc'].str.lower()
# 
    # .pivot(index = "ACCIDENT_NO", columns = 'Surface Cond Desc', values = 'SURFACE_COND')\

# pivots to create new columns for each condition
road_cond_pivotted = road_cond\
    .pivot_table(index = "ACCIDENT_NO", values = 'SURFACE_COND', aggfunc = "max")\
    .rename(columns = {"SURFACE_COND": 'dry'})\
    .fillna(0)\
    .applymap(lambda x: min(x, 1))\
    .reset_index()

# sets of interest column and outputs data
road_cond_of_interest = ['dry']
road_cond_pivotted[id_columns + road_cond_of_interest].drop_duplicates().head(3)

initial shape: (205030, 4)


,ACCIDENT_NO,dry
0,T20060000010,True
1,T20060000018,True
2,T20060000022,True


### atmospheric

after testing, same thing, simplifying atmospheric conditions made sense

In [14]:
# atmospheric condition lookup
atmosph_id_to_desc = {
    1: 'clear',
    2: 'raining',
    3: 'snowing',
    4: 'fog',
    5: 'smoke',
    6: 'dust',
    7: 'strong winds',
    9: 'not known',
}

# simplifies atmospheric into a few options
def simpilfy_atmosph_id(id):
    if id  == 1:   # clear
        return 1
    elif id in [2, 3]:    # snow or rain
        return 2
    elif id == 7:   #   wind
        return 7
    elif id in [4, 5, 6]:   #   fog_like
        return 4

In [15]:
# only keeping instances with ids that exist
# atmospheric = atmospheric[atmospheric['ACCIDENT_NO'].isin(accidents['ACCIDENT_NO'])]

# removes unknowns
atmospheric = atmospheric.query("ATMOSPH_COND != 9")
atmospheric = atmospheric.dropna(subset = ['ATMOSPH_COND'])

# simplifying conditions
atmospheric.loc[:,'ATMOSPH_COND'] = atmospheric.loc[:,'ATMOSPH_COND'].apply(simpilfy_atmosph_id)

# simplifying descriptions
atmospheric.loc[:,'Atmosph Cond Desc'] = atmospheric.loc[:,'ATMOSPH_COND'].apply(lambda id: atmosph_id_to_desc[id])

# pivots to create new columns
atmospheric_pivotted = atmospheric\
    .drop_duplicates(subset = ['ACCIDENT_NO','ATMOSPH_COND'])\
    .pivot(index = "ACCIDENT_NO", columns = 'Atmosph Cond Desc', values = 'ATMOSPH_COND')\
    .fillna(0)\
    .applymap(lambda x: min(x, 1))\
    .reset_index()

# sets of interest and prints data
atmospheric_of_interest = ['clear', 'fog', 'raining', 'strong winds']
atmospheric_pivotted[id_columns + atmospheric_of_interest].drop_duplicates().head(3)

Atmosph Cond Desc,ACCIDENT_NO,clear,fog,raining,strong winds
0,T20060000010,1.0,0.0,0.0,0.0
1,T20060000018,1.0,0.0,0.0,0.0
2,T20060000022,1.0,0.0,0.0,0.0


# combining data

In [16]:
# saving pre merged data for analysis
accidents.to_csv(ROOT_DIR + ROUGHLY_CLEANED_PRE_MERGE_DATA_DIR)

In [17]:

# merging all data with accidents, removing duplicates along the way
output = accidents[id_columns + accidents_of_interest + accidents_label_columns]\
    .drop_duplicates()\
    .merge(   
        # adding node data for location
        node[id_columns + node_of_interest].drop_duplicates(), 
        how='inner')\
    .merge(
        # adding road condition data
        road_cond_pivotted[id_columns + road_cond_of_interest].drop_duplicates(), 
        how='inner')\
    .merge(
        # adding general weather data
        atmospheric_pivotted[id_columns + atmospheric_of_interest].drop_duplicates(), 
        how='inner')\
    .merge(
        # adding person data for ambulance needed
        person_pivotted[id_columns + person_of_interest + person_label_columns].drop_duplicates(), 
        how='inner')\

# printing sample
output.head(3)

,ACCIDENT_NO,date,date_stamp,day,hour,hour_bin,season,month,day_light,police_needed,...,lga,region,lat,long,dry,clear,fog,raining,strong winds,ambulance_needed
0,T20150000056,2015-02-01 03:00:00,2015-02-01,6,3,0,1,2,0,1,...,BAW BAW,EASTERN REGION,-38.220380,145.798298,True,1.0,0.0,0.0,0.0,True
1,T20150000060,2015-02-01 07:30:00,2015-02-01,6,7,1,1,2,2,1,...,HEPBURN,WESTERN REGION,-37.341041,144.155961,True,1.0,0.0,0.0,0.0,True
2,T20150000063,2015-02-01 08:30:00,2015-02-01,6,8,1,1,2,2,1,...,SOUTHERN GRAMPIANS,SOUTH WESTERN REGION,-37.731233,142.018874,True,1.0,0.0,0.0,0.0,True


In [18]:
# printing columns to chose from
accidents_of_interest + node_of_interest + road_cond_of_interest + atmospheric_of_interest + accidents_of_interest 

['date',
 'date_stamp',
 'day',
 'hour',
 'hour_bin',
 'season',
 'month',
 'day_light',
 'node_id',
 'lga',
 'region',
 'lat',
 'long',
 'dry',
 'clear',
 'fog',
 'raining',
 'strong winds',
 'date',
 'date_stamp',
 'day',
 'hour',
 'hour_bin',
 'season',
 'month',
 'day_light']

In [19]:
# testing how it pivots
output.pivot_table(
    index = ['day','hour_bin','region','dry','clear', 'fog', 'raining', 'strong winds'],
    values = accidents_label_columns + person_label_columns,
    aggfunc = 'sum'
)

ambulance_needed  \
day hour_bin region               dry   clear fog raining strong winds                     
0   0        EASTERN REGION       False 0.0   0.0 0.0     1.0                          1   
                                                  1.0     0.0                          2   
                                                          1.0                          3   
                                              1.0 1.0     0.0                          1   
                                        1.0   0.0 0.0     0.0                          2   
...                                                                                  ...   
6   3        SOUTH WESTERN REGION True  1.0   0.0 0.0     0.0                        126   
             WESTERN REGION       False 0.0   0.0 1.0     0.0                         12   
                                                          1.0                          1   
                                        1.0   0.0 0.0     0.0                          2   
                                  True  1.0   0.0 0.0     0.0                         81   

                                                                        police_needed  
day hour_bin region               dry   clear fog raining strong winds                 
0   0        EASTERN REGION       False 0.0   0.0 0.0     1.0                       0  
                                                  1.0     0.0                       2  
                                                          1.0                       3  
                                              1.0 1.0     0.0                       1  
                                        1.0   0.0 0.0     0.0                       2  
...                                                                               ...  
6   3        SOUTH WESTERN REGION True  1.0   0.0 0.0     0.0                     110  
             WESTERN REGION       False 0.0   0.0 1.0     0.0                       9  
                                                          1.0                       1  
                                        1.0   0.0 0.0     0.0                       2  
                                  True  1.0   0.0 0.0     0.0                      70  

[1467 rows x 2 columns]

In [20]:
# outputting data for initial visuals
output.to_csv(ROOT_DIR + ROUGHLY_CLEANED_DATA_DIR)